# Script-first MCP tool debugging (Notebook)

This notebook shows the intended workflow for this repo: **debug MCP tools from notebook code**, not via a GUI.

Key idea: use `mcp_tuning.Client` as a small synchronous scripting API.


## 0) Setup (uv)

From the repo root:

```powershell
uv sync
uv sync --extra notebook --extra test
uv run jupyter lab
```


## 1) Connect to a local stdio MCP server

We'll connect to the bundled example server (FastMCP) over stdio.

Command (recommended):
- `uv run python examples/servers/simple_stdio_server.py`

Tip: you can pass either a string command or a list argv.
- List argv is safest: [".venv/Scripts/python.exe", "examples/servers/simple_stdio_server.py"]
- If you pass "uv run python ...", the helper will try to rewrite it to `.venv` python to avoid uv cache restrictions.


In [3]:
import sys, os; 
sys.executable,os.getcwd()

('D:\\tool_project\\mcp-tuning\\.venv\\Scripts\\python.exe',
 'D:\\tool_project\\mcp-tuning\\examples\\notebooks')

In [1]:
from mcp_tuning import connect_stdio

c = connect_stdio(["uv","run","python","examples/servers/simple_stdio_server.py"])
c.server_info()


{'protocolVersion': '2024-11-05',
 'capabilities': {'experimental': {},
  'prompts': {'listChanged': False},
  'resources': {'subscribe': False, 'listChanged': False},
  'tools': {'listChanged': False}},
 'serverInfo': {'name': 'simple-stdio-server', 'version': '1.25.0'}}

## 2) Explore tools/resources/prompts

These calls map to MCP endpoints like `tools/list`, `resources/list`, and `prompts/list`.


In [2]:
tools = c.tools()
[t.name for t in tools]


['echo', 'echo_json', 'add', 'now_utc']

In [3]:
c.tool('add')


ToolItem(name='add', description='Add two numbers.', input_schema={'properties': {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'})

In [4]:
c.resources()


[ResourceItem(uri='simple://readme', name='readme', description='A tiny example resource.', mime_type='text/plain')]

In [5]:
c.prompts()


[PromptItem(name='hello', description='A tiny example prompt.', arguments_schema=None)]

## 3) Call tools (debugging)

`call()` returns a `CallResult` with `ok`, `result`, `error`, and `stderr_tail` (stdio only).


In [6]:
r = c.call('add', {'a': 1, 'b': 2})
r.ok, r.result


(True,
 {'content': [{'type': 'text', 'text': '{\n  "sum": 3.0\n}'}],
  'structuredContent': {'sum': 3.0},
  'isError': False})

### Rendering tips

- `c.show(obj)` tries to render markdown/text when present, otherwise JSON.
- Use `compact=False` when you want the full payload printed (can be large).


In [7]:
c.show(r.result)


{
  "sum": 3.0
}


## 4) Save inputs as a case (tool + args)

Cases are stored under `cases/` as `*.json` and can be replayed later.


In [8]:
path = c.save_case('add', {'a': 10, 'b': 20}, note='basic add test')
path


WindowsPath('D:/tool_project/mcp-tuning/examples/notebooks/cases/add-f4dc8257ae.json')

## 5) Export logs/history/last result

Exports are written under `exports/` as JSON files.


In [9]:
c.export_logs()
c.export_history()
c.export_last()


WindowsPath('D:/tool_project/mcp-tuning/examples/notebooks/exports/result-add-20251230T151824Z.json')

## 6) Cleanup

Close the subprocess when you're done.


In [10]:
c.close()
